In [13]:
# coding:utf-8
from __future__ import print_function
from random import shuffle
from past.builtins import xrange
import pickle
import numpy as np

# 文件读取
def unpickle(file):
	with open(file,'rb') as fo:
		dict = pickle.load(fo)
	return dict

def load_file(file):
	dictTrain = unpickle(file + "data_batch_1")
	dataTrain = dictTrain['data']
	labelTrain = dictTrain['labels']

	for i in range(2,6):
		dictTrain = unpickle(file + "data_batch_" + str(i))
		dataTrain = np.vstack([dataTrain,dictTrain['data']])
		labelTrain = np.hstack([labelTrain,dictTrain['labels']])

	dictTest = unpickle(file + "test_batch")
	dataTest = dictTest['data']
	labelTest = dictTest['labels']
	labelTest = np.array(labelTest)

	return dataTrain, labelTrain, dataTest, labelTest


#softmax loss 函数
def softmax_loss_naive(W, X, y, reg):
	'''
		W:权重矩阵
		X:图片训练集(矩阵)
		y:图片训练集标签(数组)
		reg:正则化强度

		return:
			loss:训练集平均loss值
			dW:梯度矩阵
	'''
	#初始化数据
	loss = 0.0
	dW = np.zeros_like(W)
	num_train = X.shape[0]	#样本数
	num_class = W.shape[1]	#样本类别数

	for i in xrange(num_train):
		score = X[i].dot(W)
		score -= np.max(score)	#提高样本稳定性

		correct_score = score[y[i]]
		exp_sum = np.sum(np.exp(score))
		loss += np.log(exp_sum) - correct_score

		for j in xrange(num_class):
			if (j == y[i]):
				dW[:, j] += np.exp(score[j]) / exp_sum * X[i] - X[i]
			else:
				dW[:, j] += np.exp(score[j]) / exp_sum * X[i]


	loss /= num_train
	loss += 0.5 * reg * np.sum(W*W)

	dW /= num_train
	dW += reg * W

	return loss, dW

#线性分类器
class LinearClassifier(object):
	def __init__(self):
		self.W = None

	def train(self, X, y, step_size = 1e-3, reg = 1e-5, num_iters = 100, batch_size = 200, verbose = True):
		'''
		X:图片训练集(矩阵)
		y:图片训练集标签(数组)
		step_size:学习步进速度
		reg:正则化强度
		num_iters:迭代次数
		batch_size:每次迭代图片样本数
		verbose:是否打印信息

		return:
			loss_history:每次训练loss值
		'''
		num_train, dim = X.shape
		num_classes = np.max(y) + 1
	
		if self.W is None:
			self.W = 0.001 * np.random.randn(dim, num_classes)

		loss_history = []

		for it in xrange(num_iters):
			#从样本中不重复随机采batch_size个样本
			sample_index = np.random.choice(num_train, batch_size, replace=False)

			X_batch = X[sample_index, :]
			y_batch = y[sample_index]

			loss, grad = self.loss(X_batch, y_batch, reg)
			loss_history.append(loss)

			self.W += -step_size * grad

			if (verbose and it %10 == 0):
				print('iteration %d / %d, samples: %d, loss: %f' % (it, num_iters, batch_size, loss))

		return loss_history

	def predict(self, X):
		'''
		X:图片训练集(矩阵)

		return:
			y_pred:标签预测值
		'''
		y_pred = np.zeros(X.shape[1])

		score = X.dot(self.W)
		y_pred = np.argmax(score, axis = 1)

		return y_pred


	def loss(self, X_batch, y_batch, reg):
		'''
		X_batch:图片训练集(矩阵)
		y_batch:图片训练集标签(数组)
		reg:正则化强度

		return:
			loss:训练集平均loss值
			dW:梯度矩阵
		'''
		return softmax_loss_naive(self.W, X_batch, y_batch, reg)

In [20]:
from skimage.feature import local_binary_pattern
from PIL import Image
#开始训练
file_path = './'

dataTrFirst, labelTrain, dataTsFirst, labelTest = load_file(file_path)

dataTr = np.zeros((dataTrFirst.shape[0],32*32))
dataTs = np.zeros((dataTsFirst.shape[0],32*32))


for i in range(dataTrFirst.shape[0] -45000):
    img = dataTrFirst[i].reshape((32,32,3))
    img = Image.fromarray(img)
    res = local_binary_pattern(img.convert('L'),8,2,method='uniform')
    dataTr[i] = res.reshape((1,32*32))
print("训练集加载完成")

for i in range(dataTsFirst.shape[0] -1):
    img = dataTsFirst[i].reshape((32,32,3))
    img = Image.fromarray(img)
    res = local_binary_pattern(img.convert('L'),8,2,method='uniform')
    dataTs[i] = res.reshape((1,32*32))
print("测试集加载完成")

# dataTrain = dataTrain - np.mean(dataTrain, axis=0)


训练集加载完成
测试集加载完成


In [21]:
LC = LinearClassifier()

print('start training ...')
#train(self, X, y, step_size = 1e-3, reg = 1e-5, num_iters = 100, batch_size = 200, verbose = True)
#在dataTrain中不重复随机抽取batch_size个样本，迭代训练num_iters次
loss_all = LC.train(dataTrain, labelTrain, num_iters = 8000, batch_size = 200)


start training ...
iteration 0 / 8000, samples: 200, loss: 5.403901
iteration 10 / 8000, samples: 200, loss: 299.447268
iteration 20 / 8000, samples: 200, loss: 366.026709
iteration 30 / 8000, samples: 200, loss: 407.031004
iteration 40 / 8000, samples: 200, loss: 262.697126
iteration 50 / 8000, samples: 200, loss: 343.948993
iteration 60 / 8000, samples: 200, loss: 347.827261
iteration 70 / 8000, samples: 200, loss: 219.524843
iteration 80 / 8000, samples: 200, loss: 246.094681
iteration 90 / 8000, samples: 200, loss: 207.663587
iteration 100 / 8000, samples: 200, loss: 202.247361
iteration 110 / 8000, samples: 200, loss: 458.485661
iteration 120 / 8000, samples: 200, loss: 371.438184
iteration 130 / 8000, samples: 200, loss: 410.250262
iteration 140 / 8000, samples: 200, loss: 208.647782
iteration 150 / 8000, samples: 200, loss: 309.316653
iteration 160 / 8000, samples: 200, loss: 244.007702
iteration 170 / 8000, samples: 200, loss: 385.506135
iteration 180 / 8000, samples: 200, loss

iteration 1540 / 8000, samples: 200, loss: 298.460593
iteration 1550 / 8000, samples: 200, loss: 328.075856
iteration 1560 / 8000, samples: 200, loss: 275.338126
iteration 1570 / 8000, samples: 200, loss: 266.436766
iteration 1580 / 8000, samples: 200, loss: 198.095987
iteration 1590 / 8000, samples: 200, loss: 284.163846
iteration 1600 / 8000, samples: 200, loss: 249.349478
iteration 1610 / 8000, samples: 200, loss: 167.297191
iteration 1620 / 8000, samples: 200, loss: 261.833825
iteration 1630 / 8000, samples: 200, loss: 252.410352
iteration 1640 / 8000, samples: 200, loss: 205.213429
iteration 1650 / 8000, samples: 200, loss: 163.968354
iteration 1660 / 8000, samples: 200, loss: 261.044023
iteration 1670 / 8000, samples: 200, loss: 309.419829
iteration 1680 / 8000, samples: 200, loss: 216.753568
iteration 1690 / 8000, samples: 200, loss: 256.301804
iteration 1700 / 8000, samples: 200, loss: 247.468460
iteration 1710 / 8000, samples: 200, loss: 240.908192
iteration 1720 / 8000, sampl

iteration 3060 / 8000, samples: 200, loss: 295.804183
iteration 3070 / 8000, samples: 200, loss: 288.025025
iteration 3080 / 8000, samples: 200, loss: 333.325739
iteration 3090 / 8000, samples: 200, loss: 235.480404
iteration 3100 / 8000, samples: 200, loss: 369.310442
iteration 3110 / 8000, samples: 200, loss: 171.047803
iteration 3120 / 8000, samples: 200, loss: 250.931389
iteration 3130 / 8000, samples: 200, loss: 200.007943
iteration 3140 / 8000, samples: 200, loss: 171.208628
iteration 3150 / 8000, samples: 200, loss: 241.952438
iteration 3160 / 8000, samples: 200, loss: 299.817491
iteration 3170 / 8000, samples: 200, loss: 268.896882
iteration 3180 / 8000, samples: 200, loss: 434.361476
iteration 3190 / 8000, samples: 200, loss: 346.173013
iteration 3200 / 8000, samples: 200, loss: 234.682644
iteration 3210 / 8000, samples: 200, loss: 336.351911
iteration 3220 / 8000, samples: 200, loss: 261.670538
iteration 3230 / 8000, samples: 200, loss: 331.402435
iteration 3240 / 8000, sampl

iteration 4580 / 8000, samples: 200, loss: 368.821647
iteration 4590 / 8000, samples: 200, loss: 331.595567
iteration 4600 / 8000, samples: 200, loss: 225.268194
iteration 4610 / 8000, samples: 200, loss: 318.093934
iteration 4620 / 8000, samples: 200, loss: 320.204011
iteration 4630 / 8000, samples: 200, loss: 159.225722
iteration 4640 / 8000, samples: 200, loss: 395.447617
iteration 4650 / 8000, samples: 200, loss: 196.667904
iteration 4660 / 8000, samples: 200, loss: 348.125749
iteration 4670 / 8000, samples: 200, loss: 228.300768
iteration 4680 / 8000, samples: 200, loss: 246.600053
iteration 4690 / 8000, samples: 200, loss: 350.444427
iteration 4700 / 8000, samples: 200, loss: 205.410756
iteration 4710 / 8000, samples: 200, loss: 289.521489
iteration 4720 / 8000, samples: 200, loss: 273.585934
iteration 4730 / 8000, samples: 200, loss: 262.521636
iteration 4740 / 8000, samples: 200, loss: 220.736580
iteration 4750 / 8000, samples: 200, loss: 193.995148
iteration 4760 / 8000, sampl

iteration 6100 / 8000, samples: 200, loss: 179.317263
iteration 6110 / 8000, samples: 200, loss: 196.765871
iteration 6120 / 8000, samples: 200, loss: 226.735057
iteration 6130 / 8000, samples: 200, loss: 257.558722
iteration 6140 / 8000, samples: 200, loss: 307.166607
iteration 6150 / 8000, samples: 200, loss: 299.458398
iteration 6160 / 8000, samples: 200, loss: 225.930741
iteration 6170 / 8000, samples: 200, loss: 304.216165
iteration 6180 / 8000, samples: 200, loss: 180.769063
iteration 6190 / 8000, samples: 200, loss: 255.344486
iteration 6200 / 8000, samples: 200, loss: 206.285694
iteration 6210 / 8000, samples: 200, loss: 199.008152
iteration 6220 / 8000, samples: 200, loss: 264.356083
iteration 6230 / 8000, samples: 200, loss: 219.534037
iteration 6240 / 8000, samples: 200, loss: 359.111552
iteration 6250 / 8000, samples: 200, loss: 203.621596
iteration 6260 / 8000, samples: 200, loss: 287.603423
iteration 6270 / 8000, samples: 200, loss: 377.285775
iteration 6280 / 8000, sampl

iteration 7620 / 8000, samples: 200, loss: 279.698786
iteration 7630 / 8000, samples: 200, loss: 295.839502
iteration 7640 / 8000, samples: 200, loss: 259.897617
iteration 7650 / 8000, samples: 200, loss: 264.756579
iteration 7660 / 8000, samples: 200, loss: 276.965449
iteration 7670 / 8000, samples: 200, loss: 166.313638
iteration 7680 / 8000, samples: 200, loss: 269.055620
iteration 7690 / 8000, samples: 200, loss: 222.057376
iteration 7700 / 8000, samples: 200, loss: 193.301499
iteration 7710 / 8000, samples: 200, loss: 387.721494
iteration 7720 / 8000, samples: 200, loss: 180.760521
iteration 7730 / 8000, samples: 200, loss: 188.895939
iteration 7740 / 8000, samples: 200, loss: 185.371839
iteration 7750 / 8000, samples: 200, loss: 316.984170
iteration 7760 / 8000, samples: 200, loss: 182.636883
iteration 7770 / 8000, samples: 200, loss: 227.869268
iteration 7780 / 8000, samples: 200, loss: 171.533894
iteration 7790 / 8000, samples: 200, loss: 164.741326
iteration 7800 / 8000, sampl

In [23]:
dataTest = dataTest - np.mean(dataTest, axis=0)
print('last loss is %f' %(loss_all[-1]))
#开始预测
print('start predicting ...')
y_pred = LC.predict(dataTest)

hit = 0
for i in xrange(y_pred.size):
	if (y_pred[i] == labelTest[i]):
		hit += 1

print('the accuracy rate is %f ' % (hit/100))

last loss is 417.171393
start predicting ...
the accuracy rate is 24.000000 
